# Check Casimir RPC nodes

Edit `NETWORK` to select the network to check. 

In [9]:
import json
import boto3
import requests
from requests.auth import HTTPBasicAuth

NETWORK = "goerli"

if NETWORK not in ["mainnet", "goerli", "hardhat"]:
    raise ValueError(f"Invalid network: {NETWORK}")

url = f"https://nodes.casimir.co/eth/{NETWORK}"

headers = {'content-type': 'application/json'}
payload = {
    "method": "eth_syncing",
    "params": [],
    "jsonrpc": "2.0",
    "id": 0,
}
auth = None

if NETWORK in ["mainnet", "goerli"]:
    client = boto3.client('secretsmanager')
    response = client.get_secret_value(SecretId='casimir-rpc-credentials')
    secret = json.loads(response['SecretString'])
    auth = HTTPBasicAuth(secret['username'], secret['password'])

response = requests.post(url, data=json.dumps(payload), headers=headers, auth=auth)
data = response.json()

if data['result'] == False:
    print(f"Casimir {NETWORK} node is fully synced")
else:
    print(f"Casimir {NETWORK} node is syncing. Current block: {data['result']['currentBlock']}, Highest block: {data['result']['highestBlock']}")



Casimir goerli node is fully synced
